In [1]:
import random
import os
import re
import json
import pandas as pd
import lightgbm as lgbm
from datetime import datetime
from time import gmtime, strftime
from scipy import stats
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
import torch
import lightgbm
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.metrics import f1_score, mean_absolute_error, mean_squared_error
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
%matplotlib inline
import matplotlib.pyplot as plt  # Matlab-style plotting
import seaborn as sns
color = sns.color_palette()
sns.set_style('darkgrid')
import warnings
def ignore_warn(*args, **kwargs):
    pass
warnings.warn = ignore_warn #ignore annoying warning (from sklearn and seaborn)


from scipy import stats
from scipy.stats import norm, skew #for some statistics
# from gensim.models import KeyedVectors
# from gensim.scripts.glove2word2vec import glove2word2vec
# from PIL import Image
# # from PIL import ImageFile
# # ImageFile.LOAD_TRUNCATED_IMAGES = True
# from transformers import ViltProcessor, ViltModel


random_seed = 2020
random.seed(random_seed)
np.random.seed(random_seed)

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
## 读取数据
all_data = pd.read_csv('../data/feature_data_530.csv')

# feature_columns = ['Pid', 'train_type', 'label', 'mean_label']
feature_columns = ['user_fe_{}'.format(i) for i in range(399)]
feature_columns += ['loc_fe_{}'.format(i) for i in range(400)]
all_data = all_data.drop(feature_columns, axis=1)

# train_all_data = all_data[all_data['train_type'] != -1]
# submit_all_data = all_data[all_data['train_type'] == -1]
# indexxx = pd.read_csv('../data/index.csv')
# train_all_data = train_all_data.iloc[indexxx['index']]
# train_all_data = train_all_data.reset_index(drop=True)
# submit_all_data = submit_all_data.reset_index(drop=True)

# train_label_df = train_all_data[['Pid', 'label']]
# train_feature_df = train_all_data.drop(feature_columns, axis=1)

# submit_label_df = submit_all_data[['Pid', 'label']]
# submit_feature_df = submit_all_data.drop(feature_columns, axis=1)

# print(len(train_feature_df), len(submit_feature_df))
# print(len(train_label_df), len(submit_label_df))

In [7]:
## Use the quantile_transform to change the features, here change to the normal distribution
from sklearn.preprocessing import QuantileTransformer
columns = ['Title_len', 'Title_number','Alltags_len','Alltags_number', 'totalTags', 'totalFaves','totalInGroup',
               'photoCount','meanView', 'meanTags', 'meanFaves', 'followerCount','followingCount']
normal_trans = QuantileTransformer(output_distribution='normal')
all_data_normal = normal_trans.fit_transform(all_data[columns])

In [8]:
all_data_normal = pd.DataFrame(all_data_normal)
all_data_normal

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,0.739737,0.866189,0.072829,-0.199538,0.899572,0.852632,1.070745,0.132511,1.883589,1.438727,1.084741,1.635180,2.045965
1,1.368450,1.689670,2.754047,1.902873,1.204994,1.375452,1.834651,1.650782,1.049746,0.115548,0.685629,2.296162,0.902029
2,-0.451469,-0.225202,0.472073,0.510631,1.025192,1.135144,0.007527,0.075927,1.939863,1.905817,1.403730,1.414762,1.777778
3,0.008782,0.108103,-0.769761,-0.524976,1.374585,2.133124,1.489336,1.869731,0.303790,0.084144,1.081247,1.265532,1.681677
4,-0.961429,-5.199338,0.211070,0.311059,1.341123,-5.199338,0.793602,0.674258,1.960366,0.926121,-5.199338,2.106191,0.015055
...,...,...,...,...,...,...,...,...,...,...,...,...,...
486189,-1.401177,-5.199338,-0.472407,-0.106841,0.496580,1.284718,0.738089,-0.167016,1.103357,1.914918,1.823048,0.837915,1.343236
486190,0.401995,0.393847,0.332198,0.510631,-5.199338,-5.199338,-5.199338,-5.199338,-5.199338,-5.199338,-5.199338,-5.199338,-5.199338
486191,1.004131,1.064091,-0.372246,-0.524976,2.301203,1.336237,0.786175,0.544328,1.423318,2.068760,1.146688,0.989697,0.608973
486192,-0.213636,0.108103,0.991746,1.006211,1.364551,1.757616,1.476052,0.219314,1.627076,1.836801,1.781841,1.209808,1.109104


In [9]:
## Use the quantile_transform to change the features, here change to the uniform distribution
from sklearn.preprocessing import QuantileTransformer
columns = ['Title_len', 'Title_number','Alltags_len','Alltags_number', 'totalTags', 'totalFaves','totalInGroup',
               'photoCount','meanView', 'meanTags', 'meanFaves', 'followerCount','followingCount']
uniform_trans = QuantileTransformer(output_distribution='uniform')
all_data_uniform = uniform_trans.fit_transform(all_data[columns])

In [10]:
all_data_uniform = pd.DataFrame(all_data_uniform)
all_data_uniform

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,0.759760,0.805806,0.523524,0.416416,0.807201,0.809061,0.862863,0.554555,0.973374,0.922645,0.860338,0.947369,0.978970
1,0.906907,0.955956,0.997208,0.973473,0.877963,0.915486,0.967449,0.953717,0.856158,0.544301,0.745157,0.988217,0.825388
2,0.326326,0.409409,0.676155,0.698699,0.839172,0.874055,0.506507,0.533079,0.978111,0.966621,0.918754,0.921691,0.960151
3,0.500000,0.541041,0.227728,0.296797,0.910365,0.985178,0.933716,0.973985,0.616616,0.531519,0.859563,0.900234,0.953865
4,0.166166,0.000000,0.581582,0.621622,0.904520,0.000000,0.790262,0.750382,0.979261,0.816792,0.000000,0.982936,0.510511
...,...,...,...,...,...,...,...,...,...,...,...,...,...
486189,0.075576,0.000000,0.318819,0.450951,0.681659,0.902617,0.768769,0.430448,0.870042,0.967240,0.965610,0.809719,0.912451
486190,0.652653,0.651652,0.626126,0.698699,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
486191,0.832833,0.856857,0.351852,0.296797,0.987309,0.909718,0.786787,0.707255,0.921637,0.975827,0.872498,0.845202,0.733734
486192,0.415916,0.541041,0.839840,0.844344,0.908763,0.961207,0.931181,0.587619,0.947357,0.963246,0.962853,0.890499,0.874260


In [4]:
## Use the quantile_transform to change the features, here change to the norminal distribution
from sklearn.preprocessing import QuantileTransformer
columns = ['Title_len', 'Title_number','Alltags_len','Alltags_number', 'totalTags', 'totalFaves','totalInGroup',
               'photoCount','meanView', 'meanTags', 'meanFaves', 'followerCount','followingCount']

trans = QuantileTransformer(output_distribution='normal')
all_data = trans.fit_transform(all_data[columns])
all_data
## Use the quantile_transform to change the features, here change to the uniform distribution

array([[ 0.76470967,  0.88949451,  0.10145167, ...,  1.05729661,
         1.64292351,  2.04552335],
       [ 1.36670697,  1.74601652,  2.37891863, ...,  0.67045199,
         2.21809511,  0.94115753],
       [-0.4307273 , -0.22988412,  0.49270333, ...,  1.35563408,
         1.426318  ,  1.74777636],
       ...,
       [ 1.02995676,  1.09680356, -0.3487557 , ...,  1.11877592,
         1.0016797 ,  0.61220212],
       [-0.19105201,  0.11418529,  1.00382636, ...,  1.73025443,
         1.23691669,  1.13481361],
       [ 0.20396118,  0.40310812, -1.50894386, ...,  1.49932574,
         0.7156843 ,  1.34130046]])

In [5]:
all_data_pd = pd.DataFrame(all_data)
all_data_pd

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,0.764710,0.889495,0.101452,-0.203961,0.892996,0.872105,1.088181,0.163656,1.892087,1.370485,1.057297,1.642924,2.045523
1,1.366707,1.746017,2.378919,1.938804,1.196385,1.391117,1.863247,1.623598,1.041129,0.068396,0.670452,2.218095,0.941158
2,-0.430727,-0.229884,0.492703,0.515705,1.024301,1.139801,0.025322,0.102450,1.986333,1.825944,1.355634,1.426318,1.747776
3,0.012660,0.114185,-0.747859,-0.515705,1.374881,2.111815,1.505249,1.884018,0.282753,0.031514,1.052618,1.294952,1.678960
4,-0.967422,-5.199338,0.249427,0.308666,1.338938,-5.199338,0.810751,0.694128,2.009010,0.838236,-5.199338,2.064640,0.025322
...,...,...,...,...,...,...,...,...,...,...,...,...,...
486189,-1.434200,-5.199338,-0.444660,-0.114185,0.482358,1.294671,0.747859,-0.140497,1.094863,1.832500,1.752469,0.849624,1.341300
486190,0.403108,0.403108,0.359539,0.515705,-5.199338,-5.199338,-5.199338,-5.199338,-5.199338,-5.199338,-5.199338,-5.199338,-5.199338
486191,1.029957,1.096804,-0.348756,-0.515705,2.237213,1.347930,0.799083,0.566957,1.426581,1.952188,1.118776,1.001680,0.612202
486192,-0.191052,0.114185,1.003826,1.008673,1.371843,1.774051,1.490671,0.246987,1.626020,1.783550,1.730254,1.236917,1.134814
